In [2]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import joblib
import concurrent.futures
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import geometric_mean_score
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import os
from tqdm import tqdm
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import joblib
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from deslib.dcs import OLA
# from deslib.dcs import KNORA_U, KNORA_E
from deslib.des import  KNOP, METADES
from deslib.static import SingleBest, StackedClassifier
from deslib.static import StaticSelection
from imblearn.metrics import geometric_mean_score

c:\Users\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
x = np.load(r"x_all.npy")
y = np.load(r"y_all.npy")

In [4]:
x_all = np.load(r"x_all.npy")
y_all = np.load(r"y_all.npy")
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all, random_state=42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(103210, 8) (103210,)
(25803, 8) (25803,)


In [4]:
baggings = {
    'DecisionTree': DecisionTreeClassifier(criterion='gini'),
    'NaiveBayes': GaussianNB(),
    'Knn': KNeighborsClassifier(n_neighbors=7),
    'MLP': MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)
}

In [6]:
def train_model(model, x_train, y_train, model_dir, j):
    print(f"Entrou na função `train_model` - Iteração {j + 1}")
    
    # Gerar uma amostra bootstrap
    print(f"Gerando amostra bootstrap para a iteração {j + 1}")
    x_resampled, y_resampled = resample(x_train, y_train, replace=True, random_state=j)

    # Balancear a amostra usando SMOTE
    print(f"Balanceando a amostra com SMOTE para a iteração {j + 1}")
    smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=j)
    x_balanced, y_balanced = smote.fit_resample(x_resampled, y_resampled)

    # Treinar o modelo na amostra balanceada
    print(f"Treinando o modelo na iteração {j + 1}")
    model.fit(x_balanced, y_balanced)

    # Salva o modelo treinado
    model_path = os.path.join(model_dir, f'model_{j + 1}.joblib')
    print(f"Salvando o modelo treinado em {model_path}")
    joblib.dump(model, model_path)
    
    print(f"Finalizou a função `train_model` - Iteração {j + 1}")

def train_iteration(modelName, model, i, x_train, y_train):
    print(f"\nEntrou na função `train_iteration` - Modelo: {modelName}, Iteração: {i + 1}")
    
    # Criando o diretório
    model_dir = os.path.join("E:/DrebinStudy/Balanced/Bagging2", modelName)
    model_dir = os.path.join(model_dir, str(i + 1))
    print(f"Criando diretório {model_dir} para a iteração {i + 1}")
    os.makedirs(model_dir, exist_ok=True)  # Criar o diretório se não existir

    # Usar ThreadPoolExecutor para paralelizar o loop interno
    with ThreadPoolExecutor() as thread_executor:
        futures = []
        print(f"Iniciando o loop interno (100 iterações) - Modelo: {modelName}, Iteração: {i + 1}")
        for j in range(100):
            futures.append(thread_executor.submit(train_model, model, x_train, y_train, model_dir, j))
        
        # Aguardar todas as execuções do loop interno
        for future in tqdm(futures, desc=f"Model {modelName} Iteration {i + 1}", leave=False):
            future.result()  # Garante que exceções sejam capturadas

    print(f"Finalizou a função `train_iteration` - Modelo: {modelName}, Iteração: {i + 1}\n")

# Função principal para paralelizar os loops externo e intermediário
def parallel_training(baggings, x_train, y_train):
    print("Entrou na função `parallel_training`")
    
    # Paralelizar o loop intermediário e externo usando ProcessPoolExecutor
    with ProcessPoolExecutor() as process_executor:
        futures = []
        print(f"Iniciando o loop externo (modelos) e intermediário (30 iterações)")
        for modelName, model in baggings.items():
            print(f"{model}, {modelName}")
            for i in range(30):
                # Envia a tarefa do loop intermediário ao executor
                futures.append(process_executor.submit(train_iteration, modelName, model, i, x_train, y_train))

        # Aguarda todas as tarefas do loop intermediário e externo
        for future in tqdm(futures, desc="Training Models"):
            future.result()

    print("Finalizou a função `parallel_training`")

# Exemplo de chamada da função
parallel_training(baggings, x_train, y_train)


Entrou na função `parallel_training`
Iniciando o loop externo (modelos) e intermediário (30 iterações)


Training Models:   0%|          | 0/120 [00:00<?, ?it/s]


: 

: 

In [10]:
def print_number(j):
    # Criando o diretório
    print(j)
    model_dir = os.path.join("E:/DrebinStudy/Balanced/Bagging2", 'DecisionTreeClassifier')
    model_dir = os.path.join(model_dir, str(j))
    print(model_dir)
    os.makedirs(model_dir, exist_ok=True)  # Criar o diretório se não existir

    model = DecisionTreeClassifier(criterion='gini')
    
    # Aqui você precisará passar as variáveis x_train e y_train
    x_resampled, y_resampled = resample(x_train, y_train, replace=True, random_state=j)

    # Balancear a amostra usando SMOTE
    smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=j)
    x_balanced, y_balanced = smote.fit_resample(x_resampled, y_resampled)

    # Treinar o modelo na amostra balanceada
    model.fit(x_balanced, y_balanced)
    
    # Salvar o modelo treinado
    model_path = os.path.join(model_dir, f'model_{j}.joblib')
    joblib.dump(model, model_path)

# Usando ProcessPoolExecutor para evitar problemas com threads
if __name__ == "__main__":
    with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(print_number, range(1, 101))


In [6]:
modelName = 'NaiveBayes'

acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []
gmean_scores = []
mcc_scores = []
cohen_scores = []


for j in tqdm(range(30)):
    # Criando o diretorio
    model_dir = os.path.join("Bagging2", modelName)
    model_dir = os.path.join(model_dir, str(j))

    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, stratify=y, random_state=j
    )

    loaded_estimators = []
    for i in range(100):
        model_path = os.path.join(model_dir, f'model_{i+1}.joblib')
        Bagging = joblib.load(model_path)
        loaded_estimators.append(Bagging)

    # 5. Fazer a previsão combinada
    y_preds = np.array([estimator.predict(x_test) for estimator in loaded_estimators])
    previsoes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=y_preds)

    # Avaliar as métricas do modelo e armazenar na lista temporaria
    cm = confusion_matrix(y_test, previsoes)
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    geoMedia = geometric_mean_score(y_test, previsoes)
    mcc = matthews_corrcoef(y_test, previsoes)
    kappa = cohen_kappa_score(y_test, previsoes)

    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)
    gmean_scores.append(geoMedia)
    mcc_scores.append(mcc)
    cohen_scores.append(kappa)

    print(f'Repetição {i+1} {modelName} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Criar um DataFrame com as métricas
    metrics_df = pd.DataFrame({
        'Accuracy': [np.mean(acuracias)],
        'Precision': [np.mean(precisoes)],
        'Recall': [np.mean(recalls)],
        'F1_score': [np.mean(f1_scores)],
        'Roc_auc': [np.mean(roc_auc_scores)],
        'G-Mean': [np.mean(gmean_scores)],
        'MCC': [np.mean(mcc_scores)],
        'Cohen_Kappa': [np.mean(cohen_scores)],
        'Accuracy_std': [np.std(acuracias)],
        'Precision_std': [np.std(precisoes)],
        'Recall_std': [np.std(recalls)],
        'F1_score_std': [np.std(f1_scores)],
        'Roc_auc_std': [np.std(roc_auc_scores)],
        'G-Mean_std': [np.std(gmean_scores)],
        'MCC_std': [np.std(mcc_scores)],
        'Cohen_Kappa_std': [np.std(cohen_scores)],
    })

# Define the directory and file path
directory = "Metrics"
csv_path = os.path.join(directory, f'bagging{modelName}-2_metrics.csv')

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to a CSV file
metrics_df.to_csv(csv_path, index=False)

  3%|▎         | 1/30 [00:00<00:18,  1.54it/s]

Repetição 100 NaiveBayes - Acurácia: 0.74, Precisão: 0.11, Recall: 0.70, F1-score: 0.19, ROC AUC: 0.72


  7%|▋         | 2/30 [00:01<00:16,  1.68it/s]

Repetição 100 NaiveBayes - Acurácia: 0.71, Precisão: 0.11, Recall: 0.77, F1-score: 0.19, ROC AUC: 0.74


 10%|█         | 3/30 [00:01<00:16,  1.66it/s]

...

Repetição 100 NaiveBayes - Acurácia: 0.71, Precisão: 0.11, Recall: 0.77, F1-score: 0.19, ROC AUC: 0.74


 97%|█████████▋| 29/30 [00:16<00:00,  1.73it/s]

Repetição 100 NaiveBayes - Acurácia: 0.73, Precisão: 0.11, Recall: 0.76, F1-score: 0.19, ROC AUC: 0.74


100%|██████████| 30/30 [00:17<00:00,  1.72it/s]

Repetição 100 NaiveBayes - Acurácia: 0.75, Precisão: 0.11, Recall: 0.69, F1-score: 0.19, ROC AUC: 0.72
